## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [1]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from janitor import clean_names

from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler

from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

C:\Users\eloic\miniconda3\envs\mant_predictivo\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = 'C:/Users/eloic/Google Drive/EOI/TFM//01_MANT_PREDICTIVO'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'MntoMaquina.xlsx'

### Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_excel(ruta_completa, index_col = 0, header = 1)

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [5]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['horas_desde_utlimo_fallo_mms', 'humedad_mms', 'temperatura_mms']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [6]:
variables_finales = ['horas_desde_utlimo_fallo',
                   'humedad',
                   'temperatura']

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Corregir los nombres

In [7]:
df.rename(columns = {'Temperature':'temperatura',
                     'Humidity':'humedad',
                     'Hours Since Previous Failure':'horas_desde_utlimo_fallo',
                     'Failure':'fallo'}, inplace=True)

### Eliminar registros

#### Por duplicados

In [8]:
df.drop_duplicates(inplace = True)

### Balanceo

In [9]:
def sin_balanceo(df):
    variables_finales = ['horas_desde_utlimo_fallo',
                        'humedad',
                        'temperatura']
    x1 = df[variables_finales].copy()
    y1 = df['fallo'].map({'Yes':1,'No':0})

    return(x1, y1)

#### Aplicamos función balanceo

In [10]:
x,y = sin_balanceo(df)

Ya tenemos definida la x y la y final

## CREAR EL PIPELINE

### Instanciar calidad de datos

In [11]:
x.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7027 entries, 2016-01-01 01:00:00 to 2016-12-31 23:00:00
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   horas_desde_utlimo_fallo  7027 non-null   int64
 1   humedad                   7027 non-null   int64
 2   temperatura               7027 non-null   int64
dtypes: int64(3)
memory usage: 219.6 KB


#### Crear la función

In [12]:
x.columns.to_list()

['horas_desde_utlimo_fallo', 'humedad', 'temperatura']

In [13]:
def calidad_datos(df):
    temp = df.copy()
    
    var_imputar_mediana = ['horas_desde_utlimo_fallo', 'humedad', 'temperatura']
    
    def imputar_mediana(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))
        
    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)
    
    temp['temperatura'].clip(50,80, inplace=True)
    temp['humedad'].clip(60,120, inplace=True)
    temp['horas_desde_utlimo_fallo'].clip(0,1000, inplace=True)
    
    return(temp)

#### Convertirla en transformer

In [14]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [15]:
var_mms = ['temperatura','humedad','horas_desde_utlimo_fallo']
mms = MinMaxScaler()

### Crear el pipe del balanceo

#### Convertirla en transformer

In [17]:
# hacer_balanceo_Over = FunctionTransformer(balanceo_Over)

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [18]:
ct = make_column_transformer(
    (mms, var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [19]:
pipe_prepro = make_pipeline(hacer_calidad_datos,
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [21]:
modelo = XGBClassifier(n_jobs = -1, 
                       verbosity = 0,
                       learning_rate = 0.1,
                       max_depth = 20,
                       reg_alpha = 1,
                       reg_lambda = 0.1,
                       n_estimators = 100
                       )

#### Crear el pipe final de entrenamiento

In [22]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo,)

#### Guardar el pipe final de entrenamiento

In [23]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [24]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

C:\Users\eloic\miniconda3\envs\mant_predictivo\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [25]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [26]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)